In [26]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp

In [3]:
data = pd.read_csv("/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/data.csv").set_index(["timestamp","ticker"])
data

,timestamp,ticker,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,1d_ret
0,2015-01-06 09:30:00,1,1136.15,1156.75,1108.44,1138.28,1134.052643,2860436.43,1.1364,2.9079,5.7657,-0.018297
1,2015-01-06 09:30:00,2,1774.23,1885.20,1753.27,1838.34,1823.031224,6560835.70,7.2662,6.7834,8.1741,-0.023288
2,2015-01-06 09:30:00,4,63.31,64.62,62.18,63.76,63.277989,28095.12,0.7707,3.3496,4.5343,0.051118
3,2015-01-06 09:30:00,5,38.00,38.00,38.00,38.00,37.997172,0.00,0.0000,0.0000,0.0000,0.000000
4,2015-01-06 09:30:00,6,225.64,234.85,223.10,225.00,228.438278,1078647.12,0.4255,8.0562,13.2751,-0.028571
...,...,...,...,...,...,...,...,...,...,...,...,...
5357699,2021-07-01 09:30:00,688700,47.80,54.50,47.03,53.38,50.715300,64336.83,14.5494,19.2222,19.2222,-0.040395
5357700,2021-07-01 09:30:00,688777,90.21,95.08,89.32,94.42,93.250464,19001.61,4.6111,4.6288,4.6288,-0.005927
5357701,2021-07-01 09:30:00,688788,138.07,149.35,137.11,146.22,145.197985,13572.56,5.1954,7.7977,7.7977,-0.025238
5357702,2021-07-01 09:30:00,688819,44.04,45.64,43.95,45.15,45.026221,47249.01,1.8544,4.6754,4.6754,-0.022042


In [9]:
data

open     high      low    close         vwap  \
timestamp           ticker                                                    
2015-01-06 09:30:00 1       1136.15  1156.75  1108.44  1138.28  1134.052643   
                    2       1774.23  1885.20  1753.27  1838.34  1823.031224   
                    4         63.31    64.62    62.18    63.76    63.277989   
                    5         38.00    38.00    38.00    38.00    37.997172   
                    6        225.64   234.85   223.10   225.00   228.438278   
...                             ...      ...      ...      ...          ...   
2021-07-01 09:30:00 688700    47.80    54.50    47.03    53.38    50.715300   
                    688777    90.21    95.08    89.32    94.42    93.250464   
                    688788   138.07   149.35   137.11   146.22   145.197985   
                    688819    44.04    45.64    43.95    45.15    45.026221   
                    688981    59.69    62.50    59.40    61.82    61.542000   

                                volume  pct_chg  turnover  free_turnover  \
timestamp           ticker                                                 
2015-01-06 09:30:00 1       2860436.43   1.1364    2.9079         5.7657   
                    2       6560835.70   7.2662    6.7834         8.1741   
                    4         28095.12   0.7707    3.3496         4.5343   
                    5             0.00   0.0000    0.0000         0.0000   
                    6       1078647.12   0.4255    8.0562        13.2751   
...                                ...      ...       ...            ...   
2021-07-01 09:30:00 688700    64336.83  14.5494   19.2222        19.2222   
                    688777    19001.61   4.6111    4.6288         4.6288   
                    688788    13572.56   5.1954    7.7977         7.7977   
                    688819    47249.01   1.8544    4.6754         4.6754   
                    688981   777950.53   4.3376    7.1003         7.1003   

                              1d_ret  
timestamp           ticker            
2015-01-06 09:30:00 1      -0.018297  
                    2      -0.023288  
                    4       0.051118  
                    5       0.000000  
                    6      -0.028571  
...                              ...  
2021-07-01 09:30:00 688700 -0.040395  
                    688777 -0.005927  
                    688788 -0.025238  
                    688819 -0.022042  
                    688981 -0.019086  

[5357704 rows x 10 columns]

In [25]:
df_list = []
for column in data.columns:
    df = data[column].unstack()
    df = (df - df.mean(axis=0)) / df.std(axis=0)
    df = df.stack()
    df = pd.DataFrame(df,columns=[column])
    df_list.append(df)
df_list = pd.concat(df_list,axis=1)
df_list

open      high       low     close      vwap  \
timestamp           ticker                                                     
2015-01-06 09:30:00 1      -0.457906 -0.446403 -0.489909 -0.455758 -0.464910   
                    2      -1.808080 -1.719507 -1.806040 -1.744588 -1.760814   
                    4      -1.323516 -1.319505 -1.320882 -1.312682 -1.324514   
                    5      -0.255452 -0.282374 -0.228353 -0.254812 -0.256423   
                    6      -0.370697 -0.301053 -0.359653 -0.383206 -0.338112   
...                              ...       ...       ...       ...       ...   
2021-07-01 09:30:00 688700  1.575276  1.893031  1.672197  1.990755  1.826770   
                    688777 -0.005229  0.174162  0.131317  0.362594  0.250715   
                    688788 -0.119046  0.045238 -0.061821  0.080461  0.041502   
                    688819 -0.349645 -0.208647 -0.230194 -0.093891 -0.162453   
                    688981 -0.059920  0.153617  0.051657  0.246608  0.189364   

                              volume   pct_chg  turnover  free_turnover  \
timestamp           ticker                                                
2015-01-06 09:30:00 1       2.646412  0.504569  4.156351       3.971330   
                    2       5.418423  2.996307  5.443143       5.200748   
                    4       0.227458  0.214360  0.260127       0.129815   
                    5      -0.665211  0.002385 -0.641156      -0.642919   
                    6       3.144655  0.139340  3.154623       3.438077   
...                              ...       ...       ...            ...   
2021-07-01 09:30:00 688700 -0.538658 -0.167812 -0.538658      -0.538658   
                    688777 -0.189265  0.188844 -0.212635      -0.212635   
                    688788  0.218116  0.522310  0.175801       0.175801   
                    688819 -0.260359  0.310964 -0.260362      -0.260362   
                    688981  0.754814  0.263405  0.681490       0.681490   

                              1d_ret  
timestamp           ticker            
2015-01-06 09:30:00 1      -0.917183  
                    2      -1.000600  
                    4       1.363390  
                    5      -0.000641  
                    6      -0.939576  
...                              ...  
2021-07-01 09:30:00 688700 -0.917942  
                    688777 -0.159527  
                    688788 -0.576247  
                    688819 -0.747270  
                    688981 -0.646553  

[5357685 rows x 10 columns]

In [20]:
df - df.mean(axis=0)

ticker,1,2,4,5,6,7,8,9,10,11,...,688689,688690,688696,688698,688699,688700,688777,688788,688819,688981
timestamp,,,,,,,,,,,,,,,,,,,,,
2015-01-06 09:30:00,-204.688703,-1666.232975,-45.138994,-5.671259,-26.664411,-20.356291,-27.644316,6.336937,19.204804,-14.003063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07 09:30:00,-214.638703,-1640.342975,-44.848994,-5.671259,-29.844411,-20.686291,-29.664316,6.336937,19.204804,-14.703063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-08 09:30:00,-235.238703,-1682.262975,-41.598994,-5.671259,-36.204411,-21.296291,-28.854316,11.346937,19.204804,-15.393063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-09 09:30:00,-239.508703,-1674.862975,-41.758994,-5.671259,-33.024411,-19.966291,-26.474316,8.376937,19.204804,-14.833063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-12 09:30:00,-282.138703,-1771.032975,-39.688994,-5.671259,-39.064411,-20.136291,-22.884316,8.216937,19.204804,-15.693063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-25 09:30:00,1236.721297,339.337025,-26.768994,-26.251259,-61.404411,-66.996291,-90.044316,80.466937,-18.185196,-0.153063,...,8.610297,0.7,210.575926,2.021736,95.736875,2.545833,0.600753,-8.774497,-2.856944,-1.615794
2021-06-28 09:30:00,1242.321297,314.717025,-31.808994,-25.881259,-61.404411,-67.916291,-90.044316,76.916937,-18.075196,-0.653063,...,6.940297,3.7,176.125926,1.481736,83.886875,4.095833,-1.409247,-12.404497,-3.146944,-2.385794
2021-06-29 09:30:00,1273.661297,350.107025,-35.338994,-25.601259,-62.534411,-66.416291,-90.044316,84.996937,-18.075196,0.236937,...,8.100297,-3.0,202.925926,1.511736,121.256875,7.145833,1.200753,-9.704497,-3.316944,-0.295794


In [21]:
df

ticker,1,2,4,5,6,7,8,9,10,11,...,688689,688690,688696,688698,688699,688700,688777,688788,688819,688981
timestamp,,,,,,,,,,,,,,,,,,,,,
2015-01-06 09:30:00,1136.15,1774.23,63.31,38.00,225.64,80.11,111.92,69.75,79.73,33.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07 09:30:00,1126.20,1800.12,63.60,38.00,222.46,79.78,109.90,69.75,79.73,32.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-08 09:30:00,1105.60,1758.20,66.85,38.00,216.10,79.17,110.71,74.76,79.73,32.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-09 09:30:00,1101.33,1765.60,66.69,38.00,219.28,80.50,113.09,71.79,79.73,32.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-12 09:30:00,1058.70,1669.43,68.76,38.00,213.24,80.33,116.68,71.63,79.73,31.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-25 09:30:00,2577.56,3779.80,81.68,17.42,190.90,33.47,49.52,143.88,42.34,47.38,...,36.34,107.0,838.78,19.39,198.49,41.00,90.87,134.38,42.96,58.53
2021-06-28 09:30:00,2583.16,3755.18,76.64,17.79,190.90,32.55,49.52,140.33,42.45,46.88,...,34.67,110.0,804.33,18.85,186.64,42.55,88.86,130.75,42.67,57.76
2021-06-29 09:30:00,2614.50,3790.57,73.11,18.07,189.77,34.05,49.52,148.41,42.45,47.77,...,35.83,103.3,831.13,18.88,224.01,45.60,91.47,133.45,42.50,59.85


In [15]:
df_list.reset_index().groupby(["timestamp"]).min()

,ticker,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,1d_ret
timestamp,,,,,,,,,,,
2015-01-06 09:30:00,1,-0.089891,-0.089193,-0.089498,-0.088948,-0.089158,-0.280672,-3.497575,-1.112876,-0.825674,-6.600824
2015-01-07 09:30:00,1,-0.089065,-0.089764,-0.089181,-0.089550,-0.089623,-0.286367,-4.051552,-1.054232,-0.971808,-6.121724
2015-01-08 09:30:00,1,-0.089331,-0.089636,-0.089260,-0.089469,-0.089447,-0.308319,-4.490641,-0.779550,-0.849939,-5.495548
2015-01-09 09:30:00,1,-0.088867,-0.088979,-0.089191,-0.089510,-0.089234,-0.314419,-3.955527,-0.900283,-1.004929,-9.225300
2015-01-12 09:30:00,1,-0.089730,-0.090086,-0.089461,-0.089758,-0.089894,-0.261917,-4.671649,-0.883944,-0.955066,-5.600021
...,...,...,...,...,...,...,...,...,...,...,...
2021-06-25 09:30:00,1,-0.170602,-0.170496,-0.169022,-0.169427,-0.169653,-0.272056,-4.790798,-0.682111,-0.788302,-6.495068
2021-06-28 09:30:00,1,-0.169565,-0.170649,-0.169649,-0.170773,-0.170437,-0.353977,-2.386196,-0.673797,-0.777207,-6.439021
2021-06-29 09:30:00,1,-0.171074,-0.173247,-0.171222,-0.173131,-0.172525,-0.382898,-1.771391,-0.677665,-0.795627,-4.960831
